### BP to LF transition:
#### Variables
For each $e\in E$ and every time $t$:  
-  $requested(e,t)$        
-  $blocked(e,t)$  
- $triggered(e,t)$  



For each state $s\in S(bt)$ of every b-thread $bt$ and every time $t$:

    
- $at(s,t)$  

_________________________________________________________________
#### Rules

$\forall t \colon  \oplus_{e\in E}(triggered(e,t))$

$\forall t,e \colon  triggered(e,t) \Rightarrow requested(e,t) \wedge \neg blocked(e,t)$

$\forall bt, t \colon  \oplus_{s\in S(bt)}(at(s,t))$


$at(s,t) \iff ( t=0 \wedge s \in INITIAL)  \vee (at(s',t-1) \wedge triggered(e,t-1) \wedge (s',e,s)\in TRANSITIONS)$

$requested(e,t) \iff \bigvee_{s \in S} e \in REQUESTED(s) \wedge at(s,t)$

$blocked(e,t) \iff \bigvee_{s \in S} e \in BLOCKED(s) \wedge at(s,t)$


In [1]:
import os
import re
import yaml
from yaml import CLoader as Loader, CDumper as Dumper
from collections import defaultdict

### Parse BThreads yaml

In [2]:
r = re.compile("bt\d.yaml")
bthreads_files = list(filter(r.match, os.listdir('.')))
bthreads_files

In [3]:
bt_dict = dict()
bthreads = dict()
for bt_file in bthreads_files:
    with open(bt_file, 'r') as bt_yaml:
        bt_dict[bt_file.split('.')[0]] = yaml.load(bt_yaml, Loader=Loader)
# bt_dict

In [4]:
INITIALS = set()
EVENTS = set()
STATES = set()
BLOCKED = defaultdict(list)
REQUESTED = defaultdict(list)
TIME = [i for i in range(3)]
TRANSITIONS = set()
for bt in bt_dict:
    INITIALS.add(f"{bt}_{bt_dict[bt]['initial']}")
    for step in bt_dict[bt]['states']:
        step_id = f"{bt}_{step['id']}"
        for key, values in step.items():
            if key == 'id':
                STATES.add(step_id)
            if key in ('R', 'W', 'B'):
                for val in values:
                    EVENTS.add(val)
              
        if len(step['B']) > 0:
            BLOCKED[step_id].extend(step['B'])
        if len(step['R']) > 0:
            REQUESTED[step_id].extend(step['R'])
    for t in bt_dict[bt]['transitions']:
        TRANSITIONS.add((f"{bt}_state_{t['source']}",f"{bt}_state_{t['target']}",t['event']))

In [5]:
INITIALS

{'bt1_state_0', 'bt2_state_0', 'bt3_state_0'}

In [6]:
STATES 

{'bt1_state_0',
 'bt1_state_1',
 'bt1_state_2',
 'bt1_state_3',
 'bt2_state_0',
 'bt2_state_2',
 'bt2_state_3',
 'bt3_state_0',
 'bt3_state_1',
 'bt3_state_2',
 'bt3_state_3'}

In [7]:
EVENTS 

{'g1', 'g2', 'g3', 'h1', 'h2', 'h3', 'o1', 'o2', 'o3'}

In [8]:
BLOCKED 

defaultdict(list,
            {'bt3_state_1': ['o1'],
             'bt3_state_2': ['o3'],
             'bt2_state_0': ['o2'],
             'bt2_state_2': ['o1'],
             'bt2_state_3': ['o3']})

In [9]:
REQUESTED

defaultdict(list,
            {'bt1_state_0': ['g1', 'g2', 'g3'],
             'bt1_state_1': ['o1', 'o2', 'o3'],
             'bt1_state_2': ['h1', 'h2', 'h3']})

In [10]:
TIME

[0, 1, 2]

In [11]:
TRANSITIONS

{('bt1_state_0', 'bt1_state_1', 'g1'),
 ('bt1_state_0', 'bt1_state_1', 'g2'),
 ('bt1_state_0', 'bt1_state_1', 'g3'),
 ('bt1_state_1', 'bt1_state_3', 'o1'),
 ('bt1_state_1', 'bt1_state_3', 'o2'),
 ('bt1_state_1', 'bt1_state_3', 'o3'),
 ('bt1_state_2', 'bt1_state_0', 'h1'),
 ('bt1_state_2', 'bt1_state_0', 'h2'),
 ('bt1_state_2', 'bt1_state_0', 'h3'),
 ('bt2_state_0', 'bt2_state_1', 'h2'),
 ('bt2_state_0', 'bt2_state_2', 'h1'),
 ('bt2_state_0', 'bt2_state_3', 'h3'),
 ('bt3_state_0', 'bt3_state_1', 'g1'),
 ('bt3_state_0', 'bt3_state_2', 'g3'),
 ('bt3_state_0', 'bt3_state_3', 'g2')}

### Literals creation

In [12]:
import aiger
from pysat.formula import CNF

In [13]:
atoms = dict()
def get_atom(name):
    if name not in atoms:
        atoms[name] = aiger.atom(name)
    return atoms[name]
        

1. $\forall t \colon  \oplus_{e\in E}(triggered(e,t))$

In [14]:
def unique_event(events=EVENTS , time=TIME):
    triggered = [get_atom(f'triggered_{e}_at_{t}') for e in events for t in time]
    formula = False

    for t in triggered:
        c = t
        for tt in triggered:
            if t != tt:
                c = c & ~tt
        formula = c | formula
    return formula
  

2. $\forall t,e \colon  triggered(e,t) \Rightarrow requested(e,t) \wedge \neg blocked(e,t)$
 

In [15]:
def bp(events=EVENTS , time=TIME):
    literals = [(get_atom(f'requested_{e}_at_{t}'),
                  get_atom(f'blocked_{e}_at_{t}'),
                  get_atom(f'triggered_{e}_at_{t}')) for e in events for t in time]
    formula = True
    for r,b,t in literals:
        formula = t.implies(r & ~b) & formula
        
    return formula

3. $\forall bt, t \colon  \oplus_{s\in S(bt)}(at(s,t))$  

In [16]:
def one_state_at_time(states=STATES , time=TIME):
    literals = [get_atom(f'{s}_at_{t}') for s in states for t in time]
    formula = False
    for l in literals:
        c = l
        for ll in literals:
            if l != ll:
                c = c & ~ll
        formula = c | formula
    return formula

4. $at(s,t) \iff ( t=0 \wedge s \in INITIAL)  \vee (at(s',t-1) \wedge triggered(e,t-1) \wedge (s',e,s)\in TRANSITIONS)$  

In [17]:
def check_transitions(states=STATES , time=TIME, initials=INITIALS, transitions=TRANSITIONS):                   
    formula = True
    for t in time:
        for s in states:
            at_state = get_atom(f'{s}_at_{t}')
            right_hand_formula = False
            if t == 0 and s in initials:
                right_hand_formula = True
        else:
            for src, tar, e in transitions:     
                if tar == s:
                    prev_at_state = get_atom(f'{src}_at_{t-1}')
                    prev_triggered = get_atom(f'triggered_{e}_at_{t-1}')
                    right_hand_formula = (prev_at_state &  prev_triggered)  | right_hand_formula
        formula = (at_state == right_hand_formula) & formula
    return formula

5. $requested(e,t) \iff \bigvee_{s \in S} e \in REQUESTED(s) \wedge at(s,t)$

In [21]:
def check_requested(events=EVENTS, states=STATES , time=TIME):
    at_state = [(get_atom(f'{s}_at_{t}'), s) for s in states for t in time]
    requested = [(get_atom(f'requested_{e}_at_{t}'),e) for e in events for t in time]
    formula = True
    a= list()
    for req, e in requested:
        states_disjunction = False
        for state, s in at_state:
            if e in REQUESTED[s]:
                states_disjunction = state |states_disjunction
        formula = req.implies(states_disjunction) & states_disjunction.implies(req) & formula
    return formula

6. $blocked(e,t) \iff \bigvee_{s \in S} e \in BLOCKED(s) \wedge at(s,t)$  

In [22]:
def check_blocked(events=EVENTS, states=STATES , time=TIME):
    at_state = [(get_atom(f'{s}_at_{t}'), s) for s in states for t in time]
    blocked = [(get_atom(f'blocked{e}_at_{t}'),e) for e in events for t in time]
    formula = True
    a= list()
    for block, e in blocked:
        states_disjunction = False
        for state, s in at_state:
            if e in REQUESTED[s]:
                states_disjunction = state |states_disjunction
        formula = block.implies(states_disjunction) & states_disjunction.implies(block) & formula
    return formula

### Generate CNF

In [23]:
formula = (unique_event() &bp() & check_blocked() & check_requested() & check_transitions() & one_state_at_time() )
cnf = CNF(from_aiger=(formula.aig))
cnf.to_file('cnf_bp.cnf')